In [1]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.3 MB/s eta 0:00:00


In [2]:
# !pip install transformers[torch] accelerate>=0.20.1

In [3]:
!pip install transformers==4.28.0
!pip install sentencepiece
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


In [4]:
import os
import torch
import pandas as pd
import numpy as np
import transformers
import collections
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, default_data_collator
from tqdm import tqdm
from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from datasets import Dataset
from torch.cuda.amp import autocast, GradScaler

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/preprocess')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/postprocess')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/metric')

from preprocess import convert_answers, prepare_train_features, prepare_test_features
from postprocess import postprocess_qa_predictions
from metric import jaccard

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/chaii-hindi-and-tamil-question-answering/train.csv')
test_real = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/chaii-hindi-and-tamil-question-answering/test.csv')
mlqa = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/archive/mlqa_hindi.csv')
xquad = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/archive/xquad.csv')

In [6]:
data

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil
...,...,...,...,...,...,...
1109,26f356026,स्वामी निगमानन्द परमहंस (18 अगस्त 1880 - 29 नव...,स्वामी निगमानन्द परमहंस के तन्त्र गुरु कौन थे?,बामाक्षेपा,2691,hindi
1110,31179f1bb,भरत मुनि ने नाट्यशास्त्र नामक प्रसिद्ध ग्रन्थ ...,नित्यशास्त्र किसने लिखा है?,भरत मुनि,0,hindi
1111,0d35dc007,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,155,hindi
1112,7f997884d,"जलाल उद्दीन मोहम्मद अकबर () (१५ अक्तूबर, १५४२-...",मुगल सम्राट अकबर की मृत्यु किस वर्ष में हुई थी?,"२७ अक्तूबर, १६०५",46,hindi


In [7]:
test_real

,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil


In [8]:
mlqa

,context,question,answer_text,answer_start,language
0,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,एक प्रकार की रस्सी का नाम बताईए जो सेलेरी के क...,कोलेन्काइमा,2267,hindi
1,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,क्या चीज पूरी तरह से पैरेनकाइमा कोशिकाओं से बन...,पत्तियां,158,hindi
2,जाइलेम कोशिकाएं कोशिका भित्तियों की लिग्निकृत ...,जाइलेम किस प्रकार के पौधों में पाया जाता है?,ट्रेकियोफाइटों,292,hindi
3,फ्लोएम उच्चतर पौधों में आहार का संवहन करने वाल...,लेप्टोम क्या है?\n,समान कार्य करने वाला एक अधिक सरल ऊतक,969,hindi
4,"सभी हवाई अवयवों की बाह्यत्वचा, जड़ों को छोड़कर...",बाह्यत्वचा की कोशिकाएं क्या कर सकती हैं?,क्यटिन का संश्लेषण,433,hindi
...,...,...,...,...,...
5420,"सिडनी शेल्डन (11 फरवरी,1917 - 30 जनवरी 2007) ए...",सिडनी शेल्डन की राष्ट्रीयता क्या थी?,अमेरिकी,48,hindi
5421,राज्यों को काउंटियों या काउंटी-समकक्ष में विभा...,"अतीत में, सार्वजनिक शिक्षा और सार्वजनिक स्वास्...",राज्य,590,hindi
5422,"89 वें अकादमी पुरस्कार (ऑस्कर 2017) समारोह, मो...",पुरस्कारों का आयोजन किस दिन किया गया?,"26 फरवरी, 2017",141,hindi
5423,डीज़ल उत्सर्जन तरल (अंग्रेजी:Diesel exhaust fl...,डीईएफ का क्या अर्थ होता है?,(अंग्रेजी:Diesel exhaust fluid,19,hindi


In [9]:
xquad

,context,question,answer_text,answer_start,language
0,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,पैंथर्स डिफ़ेंस ने कितने अंक दिए?,308,35,hindi
1,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जेरेड एलन के पास कितने करियर सैक थे?,136,380,hindi
2,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,ल्यूक कुएक्ली ने कितने टैकल रजिस्टर किए?,118,743,hindi
3,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जोश नॉर्मन ने कितने बॉल को इंटरसेप्ट किया?,चार,90,hindi
4,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,इस सीज़न में टीम से किसने सबसे अधिक सैक रजिस्टर...,कावन शॉर्ट,169,hindi
...,...,...,...,...,...
1185,विद्युत आवेश के परिवर्तन की समय दर के रूप में ...,इलेक्ट्रोस्टैटिक और चुंबकीय बल के योग के रूप क...,इलेक्ट्रोस्टैटिक बल,328,hindi
1186,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,संरचनाओं में तनाव का कारण क्या बनता है?,तनाव टेंसर,343,hindi
1187,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,किसी वस्तु के आयतन में क्रॉस सेक्शन क्षेत्र की...,दबाव की शर्तें,118,hindi
1188,उस आयतन के लिए प्रासंगिक क्रॉस-सेक्शनल क्षेत्र...,सामान्य ताकतों से क्या जुड़ा है?,दबाव की शर्तें,118,hindi


In [10]:
# # Randomize the dataset
# data = data.sample(frac=1, random_state=1).reset_index(drop=True)

# # Split into training and test sets
# training_set = data[:1000].reset_index(drop=True)
# test_set = data[1000:].reset_index(drop=True)

# print(training_set.shape)
# print(test_set.shape)

# training_set.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/chaii-hindi-and-tamil-question-answering/train_made.csv', index = False)
# test_set.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/chaii-hindi-and-tamil-question-answering/test_made.csv', index = False)

In [11]:
## Looking into a tamil example of the data
(data.loc[data["language"]=="tamil"]).head(1)

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil


In [12]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/chaii-hindi-and-tamil-question-answering/train_made.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/chaii-hindi-and-tamil-question-answering/test_made.csv')

In [13]:
# train = pd.concat([train,xquad,mlqa], ignore_index=True)
train.head()

,id,context,question,answer_text,answer_start,language
0,d582b302a,"இந்தியாவில் 29 மாநிலங்களும், டெல்லி தேசிய தலை...",இந்திய நாட்டில் எத்தனை மாநிலங்கள் உள்ளன?,29,12,tamil
1,161e23791,मऊ उत्तर प्रदेश के मऊ जिले का मुख्यालय है। इसक...,मऊ शहर किस नदी के किनारे बसा है?,तमसा नदी,182,hindi
2,191367de3,சிலம்பம் என்பது ஒரு தடியடி தமிழர் தற்காப்புக் ...,சிலம்பம் எந்த நாட்டின் தற்காப்புக்கலை?,இந்தியா,10130,tamil
3,c196ba459,मलेरिया या दुर्वात एक वाहक-जनित संक्रामक रोग ह...,मलेरिया संक्रमण का इलाज किस दवा से किया जाता?,कुनैन,2278,hindi
4,47b292e2d,ईद उल-फ़ित्र या ईद उल-फितर (अरबी: عيد الفطر) म...,ईद किस धर्म के लोग मनाते है?,इस्लामी,212,hindi


In [14]:
train.shape

(1000, 6)

In [15]:
test.head()

,id,context,question,answer_text,answer_start,language
0,1eacbc70f,Part of a series onTamils\nHistory\n History o...,1948 க்கு முன்னர் இலங்கையை ஆண்டவர் யார்?,பிரித்தானிய,2341,tamil
1,dab6fa1cf,எத்தனால் (Ethanol) என்பது எரிநறா அல்லது வெறியம...,எத்தனாலின் மூலக்கூறு வாய்பாடு என்ன?,C2H5-OH,765,tamil
2,2da6238ff,होलकर राजवंश मल्हार राव से प्रारंभ हुआ जो १७२१...,होलकर राजवंश के आखिरी राजा कौन थे?,यशवंतराव होलकर द्वितीय,9493,hindi
3,4ab83393f,உலகக்கோப்பை காற்பந்து எனப்படுவது வெவ்வேறு நாடு...,முதல் உலகக்கோப்பை கால்பந்து போட்டியில் வென்ற அ...,உருகுவே,1455,tamil
4,ac0401666,"जामिया हमदर्द उच्च शिक्षा का एक संस्थान है, भा...",जामिया हमदर्द विश्वविद्यालय कहाँ स्थित है?,नई दिल्ली,53,hindi


In [16]:
tokenizer = AutoTokenizer.from_pretrained("deepset/xlm-roberta-large-squad2")

In [17]:
train = train.sample(frac=1, random_state=42)
train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

In [18]:
df_train = train[:-64].reset_index(drop=True)
df_valid = train[-64:].reset_index(drop=True)
train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

In [19]:
tokenized_train_ds = train_dataset.map(
    lambda example: prepare_train_features(example, tokenizer),
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_valid_ds = valid_dataset.map(
    lambda example: prepare_train_features(example, tokenizer),
    batched=True,
    remove_columns=valid_dataset.column_names
)

Map:   0%|          | 0/936 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [20]:
# tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
# tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True, remove_columns=valid_dataset.column_names)

In [21]:
model = AutoModelForQuestionAnswering.from_pretrained("deepset/xlm-roberta-large-squad2")

In [22]:
%env WANDB_DISABLED=True
args = TrainingArguments(
    f"chaii-qa",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


env: WANDB_DISABLED=True


In [23]:
data_collator = default_data_collator
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [24]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,0.223202


TrainOutput(global_step=388, training_loss=0.268026922166962, metrics={'train_runtime': 3302.5799, 'train_samples_per_second': 3.765, 'train_steps_per_second': 0.117, 'total_flos': 8648119510695936.0, 'train_loss': 0.268026922166962, 'epoch': 1.0})

In [26]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/NLP/Hindi and Tamil QA/chaii-hindi-and-tamil-question-answering/xlmroberta-finetune-data-chaii")

In [27]:
test['answers'] = test[['answer_start', 'answer_text']].apply(convert_answers, axis=1)
test_dataset = Dataset.from_pandas(test)
test_features = test_dataset.map(
    lambda example: prepare_test_features(example, tokenizer),
    batched=True,
    remove_columns=test_dataset.column_names
)
test_f = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/1431 [00:00<?, ? examples/s]

In [28]:
# # for example
# # Load the tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# # Load the saved model
# model = GPT2LMHeadModel.from_pretrained("path_to_saved_model_directory")

# # Create a Trainer instance
# trainer = Trainer(model=model, tokenizer=tokenizer)

In [29]:
raw_predictions = trainer.predict(test_f)

In [30]:
raw_predictions.predictions

(array([[  4.5266314,  -6.6654906,  -9.562581 , ...,  -8.690409 ,
          -9.528815 ,  -9.770849 ],
        [  4.9112954,  -7.2625165,  -9.596445 , ...,  -9.3414955,
         -10.337795 ,  -9.882626 ],
        [  4.8146243,  -8.398207 ,  -6.7303386, ..., -10.42255  ,
          -9.443499 ,  -9.903951 ],
        ...,
        [  5.1414533,  -9.8235655,  -9.969592 , ...,  -8.348889 ,
         -10.21293  ,  -9.833263 ],
        [  4.668018 ,  -9.819651 , -10.042858 , ...,  -9.272422 ,
         -10.235851 ,  -9.887026 ],
        [  3.813837 ,  -9.658235 , -10.086054 , ...,  -9.936544 ,
          -9.936544 ,  -9.936544 ]], dtype=float32),
 array([[  2.9057913,  -8.411702 , -10.982982 , ..., -11.2299385,
         -10.622529 , -11.826571 ],
        [  3.4184344,  -8.6261835, -11.0806265, ..., -10.461791 ,
          -9.910214 , -11.677064 ],
        [  3.4091554,  -9.795814 ,  -9.101601 , ...,  -9.941461 ,
         -11.228343 , -11.723165 ],
        ...,
        [  4.15148  , -10.865792 , -10.

In [32]:
examples = test_dataset
features = test_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [31]:
def postprocess_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]

                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [33]:
final_predictions = postprocess_predictions(test_dataset, test_features, raw_predictions.predictions)

Post-processing 114 example predictions split into 1431 features.


  0%|          | 0/114 [00:00<?, ?it/s]

In [34]:
references = [{"id": ex["id"], "answer": ex["answers"]['text'][0]} for ex in test_dataset]

In [35]:
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res

,id,answer,prediction,jaccard
0,1eacbc70f,பிரித்தானிய,\nஆங்கிலேயரின்,0.000
1,dab6fa1cf,C2H5-OH,C2H6O,0.000
2,2da6238ff,यशवंतराव होलकर द्वितीय,अहिल्याबाई होलकर,0.250
3,4ab83393f,உருகுவே,செருமனி,0.000
4,ac0401666,नई दिल्ली,नई दिल्ली,1.000
...,...,...,...,...
109,6d9a3f816,वान्देक्स,वान्देक्स,1.000
110,ba9560ca9,८.३ मिनट,८.३ मिनट,1.000
111,cc4c69225,सन ३३० ईसापूर्व,ईसापूर्व 330,0.250
112,de2d1e19a,கிமு 356 ஆம் ஆண்டு சூலை மாதம் 6,"சூலை 20, கிமு 356",0.375


In [36]:
print("The average Jaccard Score is ",res.jaccard.mean())

The average Jaccard Score is  0.5865375243664718
